In [22]:
import h5py
import numpy as np
import math

In [3]:
filenames =[
"./C1_R1v2_ID1v2.mp4.predictions.000_C1_R1v2_ID1v2.analysis_fixed.h5",
"./C1_R1v4_ID1v3.mp4.predictions.000_C1_R1v4_ID1v3.analysis_fixed.h5"]
# ,
# "./C2_R1v2_ID1v3.mp4.predictions.000_C2_R1v2_ID1v3.analysis_fixed.h5",
# "./C2_R4v1_ID4v1.mp4.predictions.000_C2_R4v1_ID4v1.analysis_fixed.h5",
# "./C3_R2v1_ID3v4.mp4.predictions.000_C3_R2v1_ID3v4.analysis_fixed.h5",
# "./C3_R1v4_ID4v1.mp4.predictions.000_C3_R1v4_ID4v1.analysis_fixed.h5",
# './C4_R2v1_ID3v2.mp4.predictions.000_C4_R2v1_ID3v2.analysis_fixed.h5',
# "./C4_R1v4_ID2v4.mp4.predictions.000_C4_R1v4_ID2v4.analysis_fixed.h5",
# "./C5_R2v1_ID2v4.mp4.predictions.000_C5_R2v1_ID2v4.analysis_fixed.h5",
# "./C5_R4v1_ID3v4.mp4.predictions.000_C5_R4v1_ID3v4.analysis_fixed.h5",
# "./C6_R1v2_ID1v3.mp4.predictions.slp.000_C6_R1v2_ID1v3.analysis_fixed.h5",
# "./C6_R4v1_ID4v1.mp4.predictions.000_C6_R4v1_ID4v1.analysis_fixed.h5"
# ]

def get_info(filename):
    with h5py.File(filename, "r") as f:
        dset_names = list(f.keys())
        locations = f["tracks"][:].T
        node_names = [n.decode() for n in f["node_names"][:]]
    return dset_names, locations, node_names

In [4]:
from scipy.interpolate import interp1d

def fill_missing(Y, kind="linear"):
    """Fills missing values independently along each dimension after the first."""

    # Store initial shape.
    initial_shape = Y.shape

    # Flatten after first dim.
    Y = Y.reshape((initial_shape[0], -1))

    # Interpolate along each slice.
    for i in range(Y.shape[-1]):
        y = Y[:, i]

        # Build interpolant.
        x = np.flatnonzero(~np.isnan(y))
        f = interp1d(x, y[x], kind=kind, fill_value=np.nan, bounds_error=False)

        # Fill missing
        xq = np.flatnonzero(np.isnan(y))
        y[xq] = f(xq)
        
        # Fill leading or trailing NaNs with the nearest non-NaN values
        mask = np.isnan(y)
        y[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), y[~mask])

        # Save slice
        Y[:, i] = y

    # Restore to initial shape.
    Y = Y.reshape(initial_shape)

    return Y

In [5]:
from scipy.signal import savgol_filter

def smooth_diff(node_loc, win=25, poly=3):
    """
    node_loc is a [frames, 2] array
    
    win defines the window to smooth over
    
    poly defines the order of the polynomial
    to fit with
    
    """
    node_loc_vel = np.zeros_like(node_loc)
    
    for c in range(node_loc.shape[-1]):
        node_loc_vel[:, c] = savgol_filter(node_loc[:, c], win, poly, deriv=1)
    
    node_vel = np.linalg.norm(node_loc_vel,axis=1)

    return node_vel

In [ ]:
class Trial: 
    video_trial = "name"

def __init__(self, filename, dset_names, locations, node_names):
    self.filename = filename
    self.dset_names = dset_names
    self.locations = locations
    self.node_names = node_names

In [8]:
#step 1: fill in missing locations for each video
def create_infodict(filenames):
    """
    iterirates through all files 
    fills missing locations for each file
    returns a dictionary with each filename as the key 
    and dset_names, locations, node_names as the value
    """
    for filename in filenames:
        dset_names, locations, node_names = get_info(filename)
        locations = fill_missing(locations)
        info[filename] = dset_names, locations, node_names

    return info

In [9]:
info_dict = create_infodict(filenames)
info_dict.keys()

info_dict['./C1_R1v2_ID1v2.mp4.predictions.000_C1_R1v2_ID1v2.analysis_fixed.h5'].get(node_names)


dict_keys(['./C1_R1v2_ID1v2.mp4.predictions.000_C1_R1v2_ID1v2.analysis_fixed.h5', './C1_R1v4_ID1v3.mp4.predictions.000_C1_R1v4_ID1v3.analysis_fixed.h5'])

In [37]:
#distance between noses
#distance between tail base to nose (2)
#angle of ear to nose (2)
# velocity of (2) thoraxes (2) noses
# totals: 9 features

# locations [frame, node, axes, mouse]
filename = './C1_R1v2_ID1v2.mp4.predictions.000_C1_R1v2_ID1v2.analysis_fixed.h5'
dset_names, locations, node_names = get_info(filename)


In [38]:
node_names

['ear', 'nose', 'tail_base', 'thorax', 'front_foot', 'rear_foot']

In [39]:
locations = fill_missing(locations)

In [40]:
print(locations.shape)

(1213, 6, 2, 2)


In [41]:
len(locations)

1213

In [42]:
(a,b) = (1, 2)
(c,d) = (3, 4)

print(a)

1


In [43]:
def get_distances_between_mice(locations, node_index):
    """
    takes in locations and node index 
    returns a list of distances between the nodes of the two mice
    """
    c_list = []
    for i in range(len(locations)):
    
        (x1,y1) = (locations[i,node_index,0,0], locations[i,node_index,1,0])
        # x , y coordinate of nose for mouse 1
        (x2, y2) = (locations[i,node_index,0,1], locations[i,node_index,1,1])
        # x and y coordinate of nose of mouse 2
        # solve for c using pythagroean theory
        a2 = (x1 - x2) ** 2 
        b2 = (y1 - y2) ** 2 
        c = math.sqrt(a2 + b2)
        c_list.append(c)
    return c_list




    


In [93]:
def get_distances_between_nodes(locations, node_index1, node_index2):
    """
    takes in locations and node indexes of the two body parts you want 
    within mice distances for 

    returns nested lists, list[0] is the distances within track1
    list[1] is the distances within track2

    """
    c_list = []
    m1_c_list = []
    m2_c_list = []
    for i in range(len(locations)):
        x1,y1 = locations[i, node_index1,0,0], locations[i,node_index1,1,0]
        # x , y coordinate of node 1 for mouse 1
        x2,y2 = locations[i,node_index2,0,0], locations[i,node_index2,1,0]
        # x, y coordiantes of node 2 for mouse 1
        x3, y3 = locations[i,node_index1,0,1], locations[i,node_index1,1,1]
        # x and y coordinate of node 1 of mouse 2
        x4, y4 = locations[i,node_index2,0,1], locations[i,node_index2,1,1]
        # solve for c using pythagroean theory
        a2 = (x1 - x2) ** 2 
        b2 = (y1 - y2) ** 2
        a2_m2 = (x3 -x4) ** 2
        b2_m2 = (y3 - y4) ** 2 
        c2 = math.sqrt(a2_m2 + b2_m2)
        c1 = math.sqrt(a2 + b2)
        m1_c_list.append(c1)
        m2_c_list.append(c2)
    c_list.append(m1_c_list)
    c_list.append(m2_c_list)
    return c_list

In [97]:
ose_to_thorax_distance = get_distances_between_nodes(locations, 1, 4)


[147.8716921522145,
 93.13882978814998,
 93.173391414287,
 134.00142610983644,
 134.2772732959077,
 184.3506344804888,
 184.230663770481,
 107.79775519586386,
 124.69920439320205,
 124.69062890976588,
 124.69220328402656,
 112.66084038622044,
 112.64162550417512,
 125.38786021605564,
 125.22316351389566,
 124.50819750935622,
 127.81039748342464,
 128.23541642876512,
 128.21711207018458,
 118.21858894774621,
 92.36666575762959,
 92.43223630303123,
 92.36613842813601,
 77.37528738215374,
 77.35841066307044,
 68.25362944133393,
 68.25840891993053,
 74.33288338133316,
 71.67098492266581,
 81.63591008242373,
 81.67199803657452,
 82.74051890927514,
 87.27628568805189,
 87.2873707918661,
 87.29656430758511,
 96.93023165312717,
 96.76756044126452,
 108.76226229849185,
 108.74453342780014,
 87.95273030122279,
 87.96554527141042,
 88.03884813960236,
 88.06043401435045,
 103.05753106526285,
 123.47686796529555,
 123.05877871532134,
 122.32422974132204,
 140.72694158002702,
 141.98915338914796,
 1

In [ ]:
def get_velocities(locations, node_index):

    node_loc_1 = locations[:,node_index,:,0]
    # node loc (x,y) of node of mouse 1
    node_loc_2 = locations[:,node_index,:,1]
    # x,y's of node of mouse 2 
    m1_vel = smooth_diff(node_loc_1)
    m2_vel = smooth_diff(node_loc_2)
    velocities = [m1_vel,m2_vel]
    return velocities 

In [36]:
def get_angles(locations, node_index_1, node_index_2):
    angles_all_mice = []
    frame, nodes, axes, mice = locations.shape
    for i in range(len(locations)):
        angles = []
        for mouse in range(mice - 1):
            (x_ear, y_ear) = (locations[:, node_index_1, 0, mouse], locations[:,node_index_1,1,mouse])
            (x_nose, y_nose) = (locations[:, node_index_2, 0,mouse], locations[:, node_index_2, 1, mouse])
            alt = abs(y_nose - y_ear)
            base = abs(x_ear - x_nose)
            angle = math.degrees(math.atan(alt/base))
            angles.append(angle)
        angles_all_mice.append(angles)
    return angles_all_mice




range(0, 1)